In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [16]:
csv = pd.read_csv('bmi.csv')
# csv

In [17]:
#正規化 1未満の数に直す なんでいるんだろう?
# これで、CSVの列に一括で処理していることになる　forループ回す必要がない
# csv['height'] = csv['height'] / 200
# csv['weight'] = csv['weight'] / 100
csv['height'] /= 200
csv['weight'] /= 100

In [4]:
# ラベルを3次元で表すための辞書 なんで3次元化がいるんだろう?
bmi_dic = {'thin':[1,0,0], 'normal':[0,1,0], 'fat':[0,0,1]}

In [5]:
# csv['label']の値に対して、lambda関数を適用して、それをcsv['label_pat']に保持する
# np.arrayはndarray型を生成し、普通の多重リストではなく、型が一定の行列でなければならず、より高速に処理される
csv['label_pat'] = csv['label'].apply(lambda x: np.array(bmi_dic[x]))
# csv

In [6]:
# テストデータをわける
test_csv = csv[15000:20000]

#　テストのパータンデータ これでweightとheightの列を結合したCSVデータになってる!
test_pat = test_csv[['weight', 'height']]

test_ans = list(test_csv['label_pat'])

# test_csv
# test_pat
# test_ans

In [7]:
# データフローグラフの作成
# まずプレースホルダ
x = tf.placeholder(tf.float32, [None, 2], name = 'x') # 身長体重のデータ Boardで見やすいように名前もつける
y_ = tf.placeholder(tf.float32, [None, 3], name = 'y_') # 答えのラベル Boardで見やすいように名前もつける

In [8]:
# 変数 Boardで見やすいようにscopeに纏める
with tf.name_scope('interface') as scope:
    W = tf.Variable(tf.zeros([2, 3]), name='W') # 重み Boardで見やすいように名前もつける
    b = tf.Variable(tf.zeros([3]), name = 'b') # バイアス Boardで見やすいように名前もつける

In [9]:
    with tf.name_scope('softmax') as scope:
        # 変数とプレースホルダを、ソフトマックス回帰にかける
        # http://developabout0309.blogspot.jp/2016/07/tensorflow-8.html
        y = tf.nn.softmax(tf.matmul(x, W) + b)
        # 最大値を1に近づけ、最小値を0に近づける関数?

In [10]:
#訓練 
with tf.name_scope('loss') as scope:
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

with tf.name_scope('training') as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(cross_entropy)

In [11]:
# 正解率
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))


In [12]:
# セッション生成
sess = tf.Session()
sess.run(tf.global_variables_initializer()) #初期化

In [13]:
# tensorboardを使う
tw = tf.summary.FileWriter('log', graph = sess.graph)

In [14]:
#　学習させる
for step in range(3500):
    i = (step * 100) % 14000
    rows = csv[1 + i : 1 + i  +100]
    x_pat = rows[['weight', 'height']]
    y_ans = list(rows['label_pat'])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict = fd)
    if step % 500 == 0:
        cre = sess.run(cross_entropy, feed_dict = fd)
        acc = sess.run(accuracy, feed_dict = fd)
        print('step = ', step, 'cre = ', cre, 'acc = ', acc)

acc = sess.run(accuracy, feed_dict = {x: test_pat, y_: test_ans})
# print('正解率', acc) 

step =  0 cre =  108.53 acc =  0.38
step =  500 cre =  51.9069 acc =  0.85
step =  1000 cre =  45.6281 acc =  0.97
step =  1500 cre =  39.5062 acc =  0.95
step =  2000 cre =  33.5967 acc =  0.95
step =  2500 cre =  31.9654 acc =  0.96
step =  3000 cre =  24.9812 acc =  0.96
